In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 64   # Number of sequences processed in parallel
block_size = 256  # Maximum context length for predictions
max_iters = 5000  # Maximum training iterations
eval_interval = 500  # Interval for evaluating loss
learning_rate = 3e-4  # Learning rate
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Check if GPU is available
eval_iters = 200
n_embd = 384  # Embedding dimension
n_head = 6    # Number of attention heads
n_layer = 6   # Number of transformer layers
dropout = 0.2  # Dropout rate

# Set a random seed for reproducibility
torch.manual_seed(1337)

# Load the input text data
with open('7-tiny-shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Create a vocabulary of unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Create mappings between characters and integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

# Define encoder and decoder functions
encode = lambda s: [stoi[c] for c in s]  # Encoder: string to list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # Decoder: list of integers to string

# Split the data into train and validation sets
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # 90% of the data for training, the rest for validation
train_data = data[:n]
val_data = data[n:]

# Function to get a batch of data
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Function to estimate the loss on train and val sets
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Define a single head of self-attention
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)   # (B, T, hs)
        q = self.query(x) # (B, T, hs)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x) # (B, T, hs)
        out = wei @ v
        return out

# Define multi-head self-attention
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

# Define feedforward layer
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

# Define a transformer block
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Define the GPT Language Model
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Create the GPT Language Model
model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"Step {iter}: Train loss {losses['train']:.4f}, Val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate text from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

10.788929 M parameters
step 0: train loss 4.2227, val loss 4.2305
step 500: train loss 1.7483, val loss 1.9048
step 1000: train loss 1.3965, val loss 1.6115
step 1500: train loss 1.2730, val loss 1.5342
step 2000: train loss 1.1908, val loss 1.5040
step 2500: train loss 1.1272, val loss 1.4939
step 3000: train loss 1.0690, val loss 1.5031
step 3500: train loss 1.0136, val loss 1.5060
step 4000: train loss 0.9660, val loss 1.5212
step 4500: train loss 0.9111, val loss 1.5480
step 4999: train loss 0.8651, val loss 1.5644

knower.

AUTOLYCUS:
Is so, she had a lord? ano, sir, that have you thus
Hold this one would do with noly that's not
Than one but mousand-fathing: 'tis not I infeast
It would feel not; whih can make me absent
To take and that dull believoAd the knew of his guilty ampossition.

ANGELO:
Yet morrow, sit mistress'd that this blood
Rng so succession'd as to rise,
And where I love him.

Shepherd:
Go hence!

Crownd:
He sies alops are alter'd, and bold,
Dain'd, in contempt bears

In [3]:
# hugging face transformer
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Create and load your model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Define a directory path where you want to save the model and tokenizer
save_directory = 'D:\Python\labs\LLM\model'

# Use save_pretrained() to save both the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

c:\Python3115\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Armaan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


('D:\\Python\\labs\\LLM\\model\\tokenizer_config.json',
 'D:\\Python\\labs\\LLM\\model\\special_tokens_map.json',
 'D:\\Python\\labs\\LLM\\model\\vocab.json',
 'D:\\Python\\labs\\LLM\\model\\merges.txt',
 'D:\\Python\\labs\\LLM\\model\\added_tokens.json')

In [4]:
from transformers import GPT2LMHeadModel

# Load your custom PyTorch model
custom_model = GPT2LMHeadModel.from_pretrained('D:\Python\labs\LLM\model')

# Save the model in the Transformers format
custom_model.save_pretrained('D:\Python\labs\LLM\model')

In [5]:
from transformers import GPT2Tokenizer

# Load your custom tokenizer
custom_tokenizer = GPT2Tokenizer.from_pretrained('D:\Python\labs\LLM\model')

# Save the tokenizer
custom_tokenizer.save_pretrained('D:\Python\labs\LLM\model')

('D:\\Python\\labs\\LLM\\model\\tokenizer_config.json',
 'D:\\Python\\labs\\LLM\\model\\special_tokens_map.json',
 'D:\\Python\\labs\\LLM\\model\\vocab.json',
 'D:\\Python\\labs\\LLM\\model\\merges.txt',
 'D:\\Python\\labs\\LLM\\model\\added_tokens.json')